## Architectural Patterns

Detect code smells and architectural patterns:
- **God Objects**: High-coupling nodes
- **Circular Dependencies**: Architecture violations
- **Dead Code**: Unreachable nodes
- **Seams**: Good integration points

In [ ]:
# Import libraries
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from collections import Counter

sys.path.insert(0, str(Path.cwd() / 'utils'))
from graph_client import GraphClient

plt.style.use('dark_background')
%matplotlib inline

print("✅ Libraries loaded")

In [ ]:
# Connect and build graph
client = GraphClient()
await client.connect()

In [ ]:
# Build directed graph
print("🔨 Building graph...")
G = await client.build_networkx_graph(exclude_stdlib=True, exclude_tests=True)

print(f"✅ Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

## Section 2: Detect God Objects

Find nodes with extremely high coupling (many incoming and outgoing edges).

In [ ]:
# Calculate total degree (in + out)
total_degree = {}
in_degree = dict(G.in_degree())
out_degree = dict(G.out_degree())

for node in G.nodes():
    total_degree[node] = in_degree.get(node, 0) + out_degree.get(node, 0)

# Find god objects (top 5%)
degree_threshold = np.percentile(list(total_degree.values()), 95)
god_objects = {k: v for k, v in total_degree.items() if v >= degree_threshold}
god_objects = dict(sorted(god_objects.items(), key=lambda x: x[1], reverse=True))

print(f"🚨 God Objects Detected: {len(god_objects)}\n")
print(f"Degree threshold: {degree_threshold:.0f}\n")

for i, (node, degree) in enumerate(list(god_objects.items())[:10], 1):
    in_d = in_degree.get(node, 0)
    out_d = out_degree.get(node, 0)
    print(f"{i}. {node}")
    print(f"   In: {in_d}, Out: {out_d}, Total: {degree}")

# Visualize
fig, ax = plt.subplots(figsize=(12, 6))
top_10_gods = dict(list(god_objects.items())[:10])
ax.barh(list(top_10_gods.keys()), list(top_10_gods.values()), color='#DC2626')
ax.set_xlabel('Total Degree (In + Out)', fontsize=12)
ax.set_title('Top 10 God Objects (High Coupling)', fontsize=14, fontweight='bold')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

## Section 3: Detect Circular Dependencies

Find cycles in the dependency graph (architectural violations).

In [ ]:
# Detect all cycles
try:
    cycles = list(nx.simple_cycles(G))
    print(f"🔄 Circular Dependencies Detected: {len(cycles)}\n")
    
    # Group by size
    cycle_sizes = Counter([len(c) for c in cycles])
    print("Cycles by length:")
    for size in sorted(cycle_sizes.keys()):
        print(f"   {size}-node cycles: {cycle_sizes[size]}")
    
    # Show shortest cycles
    print(f"\n📊 Shortest cycles (first 5):")
    shortest_cycles = sorted(cycles, key=len)[:5]
    for i, cycle in enumerate(shortest_cycles, 1):
        path = " ↔ ".join(cycle + [cycle[0]])
        print(f"{i}. {path}")
    
except Exception as e:
    print(f"Error detecting cycles: {e}")
    cycles = []

## Section 4: Detect Dead Code

Find isolated nodes with no dependencies.

In [ ]:
# Find dead code (isolated nodes)
dead_code = [node for node in G.nodes() if in_degree.get(node, 0) == 0 and out_degree.get(node, 0) == 0]

print(f"💀 Dead Code (Isolated Nodes): {len(dead_code)}\n")

if dead_code:
    print("First 20 dead code nodes:")
    for i, node in enumerate(dead_code[:20], 1):
        print(f"{i}. {node}")
else:
    print("No isolated dead code found.")

# Find leaves (only incoming edges)
leaves = [node for node in G.nodes() if in_degree.get(node, 0) > 0 and out_degree.get(node, 0) == 0]
print(f"\n🍃 Leaf Nodes (Terminal functions): {len(leaves)}")

# Find sources (only outgoing edges)
sources = [node for node in G.nodes() if in_degree.get(node, 0) == 0 and out_degree.get(node, 0) > 0]
print(f"📌 Source Nodes (Entry points): {len(sources)}")

## Section 5: Detect Seams (Loosely Coupled Junctions)

Find nodes that act as integration points with low internal coupling.

In [ ]:
# Calculate coupling ratio: (internal edges) / (total degree)
# Seams have low internal coupling but act as bridges

coupling_ratio = {}
for node in G.nodes():
    # Find neighbors of this node
    predecessors = list(G.predecessors(node))
    successors = list(G.successors(node))
    
    if len(predecessors) + len(successors) > 0:
        # Count how many neighbor pairs are connected
        internal_edges = 0
        for pred in predecessors:
            for succ in successors:
                if G.has_edge(pred, succ) or G.has_edge(succ, pred):
                    internal_edges += 1
        
        total_neighbors = len(predecessors) + len(successors)
        coupling_ratio[node] = internal_edges / total_neighbors

# Find seams (low coupling ratio, high degree)
seams = [(node, total_degree[node], coupling_ratio.get(node, 0)) 
         for node in G.nodes() 
         if total_degree[node] > np.percentile(list(total_degree.values()), 50)
         and coupling_ratio.get(node, 1) < 0.3]

seams = sorted(seams, key=lambda x: x[1], reverse=True)

print(f"🪡 Seams Detected (Good Integration Points): {len(seams)}\n")

if seams:
    print("Top 15 seams:")
    for i, (node, degree, coupling) in enumerate(seams[:15], 1):
        print(f"{i}. {node}: degree={degree}, coupling={coupling:.2f}")
else:
    print("No seams found.")

print("\n💡 Insight: Seams are good candidates for dependency injection!")

## Section 6: Architecture Quality Metrics

Calculate overall architecture quality scores.

In [ ]:
# Calculate architecture metrics
metrics = {}

# 1. Cyclomatic Complexity (number of cycles)
metrics['cyclomatic_complexity'] = len(cycles)

# 2. Dead Code Ratio
metrics['dead_code_ratio'] = len(dead_code) / G.number_of_nodes() if G.number_of_nodes() > 0 else 0

# 3. Modularity (avg distance between nodes)
try:
    avg_shortest_path = nx.average_shortest_path_length(G.to_undirected())
    metrics['avg_shortest_path'] = avg_shortest_path
except:
    metrics['avg_shortest_path'] = 0

# 4. Coupling (ratio of edges to possible edges)
max_edges = G.number_of_nodes() * (G.number_of_nodes() - 1)
metrics['coupling'] = G.number_of_edges() / max_edges if max_edges > 0 else 0

# 5. Fragmentation (percentage of isolated/dead code)
metrics['fragmentation'] = (len(dead_code) + len(leaves)) / G.number_of_nodes() if G.number_of_nodes() > 0 else 0

print("📊 Architecture Quality Metrics:\n")
print(f"  Cyclomatic Complexity (cycles): {metrics['cyclomatic_complexity']}")
print(f"  Dead Code Ratio: {metrics['dead_code_ratio']:.2%}")
print(f"  Average Shortest Path: {metrics['avg_shortest_path']:.2f}")
print(f"  Coupling Density: {metrics['coupling']:.4f}")
print(f"  Fragmentation: {metrics['fragmentation']:.2%}")

# Quality score (0-100)
quality_score = 100
quality_score -= min(metrics['cyclomatic_complexity'] * 5, 30)  # Cycles: -5 per cycle, max -30
quality_score -= metrics['dead_code_ratio'] * 20  # Dead code: -20 max
quality_score -= metrics['coupling'] * 30  # Coupling: -30 max
quality_score -= metrics['fragmentation'] * 10  # Fragmentation: -10 max

print(f"\n🏆 Overall Quality Score: {quality_score:.1f}/100")

if quality_score >= 80:
    print("   Grade: A (Excellent)")
elif quality_score >= 70:
    print("   Grade: B (Good)")
elif quality_score >= 60:
    print("   Grade: C (Fair)")
else:
    print("   Grade: D (Needs Improvement)")

## Summary

This notebook identified architectural patterns and code smells:

1. ✅ **God Objects** - High coupling nodes
2. ✅ **Circular Dependencies** - Architectural violations
3. ✅ **Dead Code** - Isolated unreachable nodes
4. ✅ **Seams** - Loosely coupled integration points
5. ✅ **Quality Metrics** - Overall architecture assessment

### Refactoring Recommendations:
- Break up god objects using Facade or Adapter patterns
- Resolve cycles with Dependency Inversion Principle
- Remove or integrate dead code
- Use seams for dependency injection

### Next Step:
- **Notebook 05**: Ontology Extraction (domain concepts)